## Tensorflow Tests

In [1]:
%%capture
import numpy as np
import tensorflow as tf
from torch import from_numpy as t
import network_moments.torch.gaussian as tnm
from  network_moments.tensorflow.utils import *
import network_moments.tensorflow.gaussian as fnm

eager = False

if eager:
    tf.enable_eager_execution()
else:
    tf.InteractiveSession()

In [2]:
m = 5
n = 3
p = 2
batch = 7

# numpy tensors
nA = np.random.rand(m, n)
nc1 = np.random.rand(m)
nB = np.random.rand(p, m)
nc2 = np.random.rand(p)
nmu = np.random.rand(batch, n)
nvar = np.random.rand(batch, n)
ncov = np.random.rand(batch, n, n)

# tensorflow tensors
fA = tf.constant(nA, tf.float32)
fc1 = tf.constant(nc1, tf.float32)
fB = tf.constant(nB, tf.float32)
fc2 = tf.constant(nc2, tf.float32)
fmu = tf.constant(nmu, tf.float32)
fvar = tf.constant(nvar, tf.float32)
fcov = tf.constant(ncov, tf.float32)

# torch tensors
tA = t(nA)
tc1 = t(nc1)
tB = t(nB)
tc2 = t(nc2)
tmu = t(nmu)
tvar = t(nvar)
tcov = t(ncov)

In [3]:
def comp(a, b):
    res = tf.reduce_max(tf.abs(a - b))
    if tf.executing_eagerly():
        return res.numpy()
    else:
        return res.eval()

print('# gaussian.affine:')
print("mean():", comp(tnm.affine.mean(tmu, tA, tc1).numpy(),
                      fnm.affine.mean(fmu, fA, fc1)))
print("covariance():", comp(tnm.affine.covariance(tcov, tA).numpy(),
                            fnm.affine.covariance(fcov, fA)))
print("variance():", comp(tnm.affine.variance(tcov, tA).numpy(),
                          fnm.affine.variance(fcov, fA)))

print('# gaussian.relu:')
print("mean():", comp(tnm.relu.mean(tmu, tvar).numpy(),
                      fnm.relu.mean(fmu, fvar)))
print("covariance():", comp(tnm.relu.zero_mean_covariance(tcov).numpy(),
                          fnm.relu.zero_mean_covariance(fcov)))

print('# gaussian.affine_relu_affine:')
print("mean():", comp(tnm.affine_relu_affine.mean(tmu, tcov, tA, tc1, tB, tc2).numpy(),
                      fnm.affine_relu_affine.mean(fmu, fcov, fA, fc1, fB, fc2)))
print("covariance():", comp(tnm.affine_relu_affine.special_covariance(tcov, tA, tB).numpy(),
                            fnm.affine_relu_affine.special_covariance(fcov, fA, fB)))
print("variance():", comp(tnm.affine_relu_affine.special_variance(tcov, tA, tB).numpy(),
                          fnm.affine_relu_affine.special_variance(fcov, fA, fB)))

if not eager:
    print('# Linearization')
    ins = tf.placeholder(tf.float32, shape=(None, n))
    out = tf.maximum(matmul(ins, fA, transpose_b=True)+fc1, 0)
    A, c1 = linearize(out, ins, nmu)
    print('A:', comp(nA, A))
    print('c1:', comp(nc1, c1))

# gaussian.affine:
mean(): 1.1920929e-07
covariance(): 4.7683716e-07
variance(): 4.7683716e-07
# gaussian.relu:
mean(): 5.9604645e-08
covariance(): 8.940697e-08
# gaussian.affine_relu_affine:
mean(): 4.7683716e-07
covariance(): 9.536743e-07
variance(): 9.536743e-07
# Linearization
A: 1.7274828234548067e-08
c1: 1.3130023945873148e-07
